In [1]:
pip install --upgrade selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pdfplumber PyPDF2

In [4]:
pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install html5lib lxml

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Importation des bibliothèques
import requests
from io import StringIO
import logging
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pdfplumber
import os
import time
import pandas as pd
from pymongo import MongoClient

In [18]:
# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'scraping_2024_2025_{datetime.now().strftime("%Y%m%d")}.log'),
        logging.StreamHandler()
    ]
)

# Dossier pour stocker les données
data_folder = "C:/Users/21655/bct_data_clean_2024_2025"
output_folder = "C:/Users/21655/bct_data_clean_2024_2025/processed"
os.makedirs(data_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Fonction de nettoyage des DataFrames
def clean_dataframe(df):
    logging.info(f"Tableau avant nettoyage : {df.shape}, colonnes : {df.columns.tolist()}")
    df = df.dropna(how='all').copy()
    for col in df.columns:
        if df[col].dtype == "object":
            df.loc[:, col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)
    df.columns = [f"col_{i}" if str(col).strip() in ('', 'Unnamed') 
                 else str(col).strip() 
                 for i, col in enumerate(df.columns)]
    logging.info(f"Tableau après nettoyage : {df.shape}, colonnes : {df.columns.tolist()}")
    return df

# Fonction pour extraire les tableaux des PDF avec pdfplumber
def extract_tables_from_pdf(pdf_path, filename_prefix):
    try:
        saved_files = []
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages):
                tables = page.extract_tables()
                for i, table in enumerate(tables):
                    if not table:
                        continue
                    df = pd.DataFrame(table[1:], columns=table[0] if table[0] else None)
                    df = clean_dataframe(df)
                    if df.empty or len(df) < 1 and len(df.columns) < 2:
                        logging.warning(f"Tableau {i} (page {page_num+1}) ignoré dans {pdf_path} (vide ou trop petit: {df.shape})")
                        continue
                    filename = f"{filename_prefix}_pdf_table_page_{page_num+1}_{i}.csv"
                    filepath = os.path.join(output_folder, filename)
                    df.to_csv(filepath, index=False, encoding='utf-8-sig')
                    logging.info(f"Sauvegardé {filename} ({len(df)} lignes)")
                    saved_files.append({
                        'Statistique': filename_prefix,
                        'Fichier': filename,
                        'Type': 'PDF Table',
                        'Ligne_echantillon': df.iloc[0].to_dict() if not df.empty else None
                    })
        return saved_files
    except Exception as e:
        logging.error(f"Erreur extraction PDF {pdf_path}: {e}")
        return []

# Fonction pour scraper les tableaux HTML
def scrape_and_save_tables(driver, data_folder, link_info):
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        valid_tables = []
        for table in soup.find_all('table'):
            if len(table.find_all(['tr', 'td', 'th'])) >= 3:
                valid_tables.append(table)
        
        if not valid_tables:
            logging.warning(f"Aucun tableau valide trouvé dans {link_info['Nom']}")
            return []
        
        saved_files = []
        for i, table in enumerate(valid_tables):
            try:
                logging.debug(f"HTML du tableau {i} : {str(table)[:500]}")
                for tag in table(['span', 'div', 'script', 'style', 'a']):
                    tag.decompose()
                
                try:
                    with StringIO(str(table)) as buffer:
                        df = pd.read_html(buffer, flavor='bs4')[0]
                except Exception as e:
                    logging.warning(f"read_html failed: {e}")
                    rows = []
                    headers = None
                    for tr in table.find_all('tr'):
                        cols = [td.get_text(strip=True) for td in tr.find_all(['th', 'td'])]
                        if cols and len(cols) > 1:
                            if not headers and tr.find('th'):
                                headers = cols
                            else:
                                rows.append(cols)
                    df = pd.DataFrame(rows, columns=headers) if rows and headers else pd.DataFrame()
                
                df = clean_dataframe(df)
                if df.empty or (len(df) < 1 and len(df.columns) < 2):
                    logging.warning(f"Tableau {i} ignoré dans {link_info['Nom']} (vide ou trop petit: {df.shape}), contenu : {df.to_dict()}")
                    continue
                
                unnamed_cols = [col for col in df.columns if 'Unnamed' in str(col)]
                if len(unnamed_cols) > len(df.columns) / 2:
                    logging.warning(f"Tableau {i} dans {link_info['Nom']} a trop de colonnes Unnamed: {unnamed_cols}")
                    continue
                
                safe_name = "".join(c if c.isalnum() else "_" for c in link_info['Nom'])
                filename = f"{safe_name}_2024_2025_table_{i}.csv"
                filepath = os.path.join(data_folder, filename)
                
                df.to_csv(filepath, index=False, encoding='utf-8-sig')
                
                if os.path.getsize(filepath) < 50:
                    logging.warning(f"Fichier {filename} petit mais conservé pour analyse")
                
                logging.info(f"Sauvegardé {filename} ({len(df)} lignes)")
                saved_files.append({
                    'Statistique': link_info['Nom'],
                    'Fichier': filename,
                    'Type': 'HTML Table',
                    'Ligne_echantillon': df.iloc[0].to_dict() if not df.empty else None
                })
                
            except Exception as e:
                logging.error(f"Erreur tableau {i} dans {link_info['Nom']}: {e}")
                continue
        
        return saved_files
    
    except Exception as e:
        logging.error(f"ERREUR dans {link_info['Nom']}: {e}")
        return []

# Fonction pour récupérer les liens des statistiques
def get_statistics_links(driver):
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//table//tr[td[@colspan='3']]"))
        )
        
        links = driver.find_elements(By.XPATH, "//table//tr[not(td[@colspan='3'])]/td[1]/a")
        links_data = []
        for link in links:
            name = link.text.strip()
            if any(keyword in name.lower() for keyword in [
                'transactions de change', 'taux de change', 'banques', 
                'secteur monétaire', 'situation mensuelle', 'taux d’intérêt', 
                '2024', '2025'
            ]):
                links_data.append({
                    "Nom": name,
                    "Lien": link.get_attribute('href')
                })
        return links_data
    except Exception as e:
        logging.error(f"Erreur récupération des liens : {e}")
        return []

# Fonction alternative avec requests et BeautifulSoup (si Selenium échoue)
def scrape_with_requests(url):
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        logging.info("Page chargée avec succès via requests !")
    except requests.exceptions.Timeout:
        logging.error("Timeout lors du chargement de la page via requests. Réessayez plus tard.")
        return None, []
    except requests.exceptions.RequestException as e:
        logging.error(f"Erreur lors de la requête : {e}")
        return None, []
    
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    if not table:
        logging.error("Aucun tableau trouvé sur la page.")
        return None, []
    
    rows = []
    headers = None
    for tr in table.find_all('tr'):
        cols = [td.get_text(strip=True) for td in tr.find_all(['th', 'td'])]
        if not cols:
            continue
        if not headers:
            headers = cols
        else:
            rows.append(cols)
    
    df = pd.DataFrame(rows, columns=headers)
    logging.info(f"Tableau extrait via requests : {df.shape}, colonnes : {df.columns.tolist()}")
    
    links = []
    for row in table.find_all('tr'):
        link = row.find('a')
        if link:
            links.append({
                "Nom": link.get_text(strip=True),
                "Lien": link['href']
            })
    
    return df, links

# Main script
def main():
    # Initialisation du driver Selenium
    path = "C:/chromedriver.exe"
    service = Service(executable_path=path)
    driver = webdriver.Chrome(service=service)
    
    # URL cible
    url = "https://www.bct.gov.tn/bct/siteprod/index.jsp"
    stats_url = "https://www.bct.gov.tn/bct/siteprod/statistiques.jsp?la=fr"
    
    # Étape 1 : Scraper le tableau de périodicité avec requests
    logging.info("Tentative de scraping avec requests...")
    periodicity_df, links = scrape_with_requests(stats_url)
    if periodicity_df is not None:
        periodicity_df.to_csv(os.path.join(data_folder, "periodicity_table.csv"), index=False, encoding='utf-8-sig')
        logging.info("Tableau de périodicité sauvegardé dans 'periodicity_table.csv'")
        pd.DataFrame(links).to_csv(os.path.join(data_folder, "links_periodicity.csv"), index=False)
        logging.info("Liens sauvegardés dans 'links_periodicity.csv'")
    else:
        logging.warning("Échec du scraping avec requests. Tentative avec Selenium...")
        
        # Étape 2 : Charger la page avec Selenium (avec gestion de timeout)
        driver.set_page_load_timeout(300)  # Timeout de 5 minutes
        try:
            driver.get(url)
            logging.info("Page principale chargée avec succès !")
        except TimeoutException as e:
            logging.error(f"Timeout lors du chargement de {url}: {e}")
            logging.info("Nouvelle tentative dans 10 secondes...")
            time.sleep(10)
            try:
                driver.get(url)
                logging.info("Page chargée avec succès à la deuxième tentative !")
            except Exception as e:
                logging.error(f"Échec définitif du chargement de {url}: {e}")
                driver.quit()
                return
        
        # Cliquer sur la section Statistiques
        try:
            stats_link = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Statistiques')]"))
            )
            driver.execute_script("arguments[0].click();", stats_link)
            logging.info("Bouton 'Statistiques' cliqué avec JavaScript !")
        except Exception as e:
            logging.error(f"Erreur lors du clic sur Statistiques : {e}")
            driver.quit()
            return
        
        # Récupérer les liens
        links = get_statistics_links(driver)
        pd.DataFrame(links).to_csv(os.path.join(data_folder, "links_statistiques_2024_2025.csv"), index=False)
        logging.info("Liens sauvegardés dans 'links_statistiques_2024_2025.csv'")
    
    # Étape 3 : Traiter chaque lien
    all_data = []
    for i, link in enumerate(links, 1):
        logging.info(f"Traitement {i}/{len(links)}: {link['Nom']}")
        try:
            if not link['Lien'].startswith('http'):
                link['Lien'] = f"https://www.bct.gov.tn{link['Lien']}"
            
            driver.set_page_load_timeout(300)
            driver.get(link['Lien'])
            
            # Chercher les liens PDF
            pdf_links = []
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            for a in soup.find_all('a', href=True):
                if 'pdf' in a['href'].lower():
                    pdf_links.append(a['href'])
            
            # Télécharger les PDF
            if pdf_links:
                for pdf_url in pdf_links:
                    if not pdf_url.startswith('http'):
                        pdf_url = f"https://www.bct.gov.tn{pdf_url}"
                    pdf_response = requests.get(pdf_url, timeout=30)
                    pdf_filename = pdf_url.split('/')[-1]
                    pdf_filepath = os.path.join(data_folder, pdf_filename)
                    with open(pdf_filepath, 'wb') as f:
                        f.write(pdf_response.content)
                    logging.info(f"PDF téléchargé : {pdf_filename}")
                    pdf_results = extract_tables_from_pdf(pdf_filepath, link['Nom'])
                    all_data.extend(pdf_results)
            
            # Scraper les tableaux HTML
            html_results = scrape_and_save_tables(driver, data_folder, link)
            all_data.extend(html_results)
            
        except Exception as e:
            logging.error(f"Erreur traitement {link['Nom']}: {e}")
            continue
    
    # Étape 4 : Traiter les PDF déjà téléchargés
    logging.info("Traitement des PDF déjà téléchargés...")
    for pdf_file in os.listdir(data_folder):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(data_folder, pdf_file)
            filename_prefix = pdf_file.replace('.pdf', '')
            logging.info(f"Traitement du PDF : {pdf_file}")
            pdf_results = extract_tables_from_pdf(pdf_path, filename_prefix)
            all_data.extend(pdf_results)
    
    # Sauvegarder les métadonnées
    if all_data:
        pd.DataFrame(all_data).to_csv(os.path.join(data_folder, "metadata_2024_2025.csv"), index=False, encoding='utf-8-sig')
        logging.info("Métadonnées sauvegardées dans 'metadata_2024_2025.csv'")
    
    # Fermer le driver
    driver.quit()
    logging.info("Scraping terminé.")

# Exécuter le script
if __name__ == "__main__":
    main()

2025-05-01 16:13:36,756 - INFO - Tentative de scraping avec requests...
2025-05-01 16:13:36,908 - ERROR - Erreur lors de la requête : 404 Client Error: Not Found for url: https://www.bct.gov.tn/bct/siteprod/statistiques.jsp?la=fr
2025-05-01 16:13:36,909 - WARNING - Échec du scraping avec requests. Tentative avec Selenium...
2025-05-01 16:13:39,714 - INFO - Page principale chargée avec succès !
2025-05-01 16:13:40,392 - INFO - Bouton 'Statistiques' cliqué avec JavaScript !
2025-05-01 16:13:40,808 - INFO - Liens sauvegardés dans 'links_statistiques_2024_2025.csv'
2025-05-01 16:13:40,809 - INFO - Traitement 1/6: Situation mensuelle de la BCT
2025-05-01 16:13:43,433 - INFO - PDF téléchargé : 2024_Situation_Mensuelle_BCT_fr.pdf
2025-05-01 16:13:43,609 - INFO - Tableau avant nettoyage : (25, 13), colonnes : ['Indicateurs(cid:9)', '2025-01', '2025-02', '', '', '', '', '', '', '', '', '', '']
2025-05-01 16:13:43,620 - ERROR - Erreur extraction PDF C:/Users/21655/bct_data_clean_2024_2025\2024_S

In [74]:
import pdfplumber
import pandas as pd
import re
import numpy as np
import csv
from IPython.display import display, HTML

# Fonction pour vérifier si une chaîne est une date au format YYYY-MM
def is_valid_date(value):
    if not value or not isinstance(value, str):
        return False
    return bool(re.match(r"^\d{4}-\d{2}$", value.strip()))

# Fonction pour extraire l'année à partir d'une liste de dates
def get_year_from_dates(year_months):
    if not year_months:
        return None
    first_date = year_months[0]
    return first_date.split('-')[0]

# Fonction pour nettoyer les valeurs avant conversion en nombre
def clean_value(val):
    if val is None or (isinstance(val, str) and not val.strip()):
        print(f"Clean_value - Valeur vide détectée : '{val}' → NaN")
        return np.nan
    
    val = str(val).strip()
    if val.lower() in ['-', '--', '', ' ', '\n', 'n/a', 'null', 'nan', '']:
        print(f"Clean_value - Valeur non numérique détectée : '{val}' → NaN")
        return np.nan
    
    val = val.replace(" ", "").replace("\n", "").replace("\t", "").replace(",", "")
    val = re.sub(r"[^0-9.-]", "", val)
    
    if not val:
        print(f"Clean_value - Valeur vide après nettoyage : '{val}' → NaN")
        return np.nan
    
    if val.count('.') > 1 or val.count('-') > 1 or val == '-' or val == '.':
        print(f"Clean_value - Format de nombre invalide : '{val}' → NaN")
        return np.nan
    
    try:
        result = float(val)
        print(f"Clean_value - Valeur convertie : '{val}' → {result}")
        return result
    except ValueError:
        print(f"Clean_value - Impossible de convertir en nombre : '{val}' → NaN")
        return np.nan

# Fonction pour extraire les données d'un tableau
def extract_table_data(table, table_num):
    print(f"\nExtract_table_data - Début du traitement du tableau {table_num}")
    
    print(f"Extract_table_data - Nombre total de lignes dans le tableau : {len(table)}")
    for row_idx, row in enumerate(table):
        print(f"Extract_table_data - Ligne brute {row_idx} : {row}")
    
    headers = table[0]
    year_months = [header.strip() for header in headers[1:] if header]
    print(f"Extract_table_data - En-têtes détectés : {year_months}")
    
    if not all(is_valid_date(date) for date in year_months):
        print(f"Tableau {table_num} ignoré : colonnes temporelles invalides : {year_months}")
        return None

    data = {}
    for row_idx, row in enumerate(table[1:], start=1):
        print(f"Extract_table_data - Traitement de la ligne {row_idx} : {row}")
        
        if not any(cell for cell in row if cell is not None and str(cell).strip()):
            print(f"Extract_table_data - Ligne {row_idx} ignorée (entièrement vide) : {row}")
            continue
        
        indicator = row[0].strip() if row[0] else f"Indicateur_{row_idx}"
        indicator = indicator.replace(" ", "_").replace(":", "").replace("'", "_").replace("é", "e").replace("à", "a").replace("-", "_")
        print(f"Extract_table_data - Indicateur extrait (ligne {row_idx}) : {indicator}")
        
        values = [clean_value(val) for val in row[1:]]
        while len(values) < len(year_months):
            values.append(np.nan)
        values = values[:len(year_months)]
        print(f"Extract_table_data - Valeurs extraites pour {indicator} : {values}")
        
        data[indicator] = values
    
    if not data:
        print(f"Tableau {table_num} ignoré : aucune donnée valide extraite")
        return None
    
    df = pd.DataFrame(data, index=year_months)
    print(f"Tableau {table_num} - DataFrame créé avec index : {df.index.tolist()}")
    print(f"Tableau {table_num} - Colonnes extraites : {df.columns.tolist()}")
    print(f"Tableau {table_num} - Aperçu du DataFrame : \n{df}")
    
    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            print(f"Erreur : La colonne {col} contient des valeurs non numériques : {df[col].values}")
            raise ValueError(f"La colonne {col} contient des valeurs non numériques.")
    
    return df

# Fonction pour fusionner les tableaux fractionnés entre les pages
def merge_tables(tables):
    merged_tables = []
    table_num = 1
    current_table = None
    current_year = None

    for table_idx, table in enumerate(tables):
        if len(table) < 1:
            print(f"Merge_tables - Tableau {table_idx} ignoré (vide) : {table}")
            continue
            
        headers = table[0]
        year_months = [header.strip() for header in headers[1:] if header]
        print(f"Merge_tables - Vérification tableau {table_idx} - En-têtes : {year_months}")
        print(f"Merge_tables - Tableau {table_idx} - Contenu brut : {table}")

        if all(is_valid_date(date) for date in year_months):
            year = get_year_from_dates(year_months)
            print(f"Merge_tables - Tableau {table_idx} - Année détectée : {year}")

            if current_table is not None and current_year != year:
                headers = current_table[0]
                year_months = [header.strip() for header in headers[1:] if header]
                expected_length = len(year_months) + 1
                adjusted_table = []
                adjusted_table.append(current_table[0])
                for row in current_table[1:]:
                    while len(row) < expected_length:
                        row.append('')
                    row = row[:expected_length]
                    adjusted_table.append(row)
                merged_tables.append(adjusted_table)
                print(f"Merge_tables - Tableau {table_num} complet ({len(adjusted_table)} lignes) ajouté. Période : {year_months[0]} à {year_months[-1]}")
                table_num += 1

            current_table = table
            current_year = year
        else:
            if current_table is None:
                print(f"Merge_tables - Tableau {table_idx} ignoré : aucun tableau en cours pour ajouter les lignes")
                continue
            print(f"Merge_tables - Ajout des lignes au tableau en cours pour l'année {current_year}")
            # Vérifier que les lignes ont au moins une valeur non vide avant de les ajouter
            for row in table:
                if any(cell for cell in row if cell is not None and str(cell).strip()):
                    current_table.append(row)
                else:
                    print(f"Merge_tables - Ligne ignorée (entièrement vide) : {row}")

    if current_table is not None:
        headers = current_table[0]
        year_months = [header.strip() for header in headers[1:] if header]
        expected_length = len(year_months) + 1
        adjusted_table = []
        adjusted_table.append(current_table[0])
        for row in current_table[1:]:
            while len(row) < expected_length:
                row.append('')
            row = row[:expected_length]
            adjusted_table.append(row)
        merged_tables.append(adjusted_table)
        print(f"Merge_tables - Tableau {table_num} complet ({len(adjusted_table)} lignes) ajouté. Période : {year_months[0]} à {year_months[-1]}")

    return merged_tables

# Fonction pour extraire toutes les données du PDF
def extract_all_data_from_pdf(pdf_path):
    all_data = []
    raw_tables = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            tables = page.extract_tables()
            print(f"Extract_all_data - Page {page_num} : {len(tables)} tableaux détectés")
            for table_num, table in enumerate(tables, 1):
                if len(table) < 1:
                    print(f"Extract_all_data - Tableau {table_num} (page {page_num}) ignoré : vide")
                    continue
                raw_tables.append(table)
                print(f"Extract_all_data - Tableau {table_num} extrait de la page {page_num} (nombre de lignes : {len(table)})")
                print(f"Extract_all_data - Aperçu du tableau {table_num} : {table}")

    merged_tables = merge_tables(raw_tables)
    print(f"Extract_all_data - Nombre total de tableaux complets : {len(merged_tables)}")

    for table_num, table in enumerate(merged_tables, 1):
        try:
            df = extract_table_data(table, table_num)
            if df is not None:
                print(f"Extract_all_data - Tableau {table_num} - DataFrame ajouté : \n{df}")
                all_data.append(df)
        except Exception as e:
            print(f"Extract_all_data - Erreur lors de l'extraction du tableau {table_num} : {e}")
            continue
    
    if not all_data:
        raise ValueError("Aucune donnée extraite du PDF.")
    print(f"Extract_all_data - Nombre de DataFrames à fusionner : {len(all_data)}")
    merged_df = pd.concat(all_data)
    print(f"Extract_all_data - DataFrame fusionné - Index avant suppression des doublons : {merged_df.index.tolist()}")
    merged_df = merged_df[~merged_df.index.duplicated(keep='first')]
    print(f"Extract_all_data - DataFrame fusionné - Index après suppression des doublons : {merged_df.index.tolist()}")

    print(f"Extract_all_data - DataFrame fusionné - Aperçu : \n{merged_df}")

    merged_df.index = pd.to_datetime(merged_df.index, format="%Y-%m")
    merged_df = merged_df.sort_index()
    merged_df.index = merged_df.index.strftime("%Y-%m")
    return merged_df

# Chemin vers le fichier PDF
pdf_path = "C:\\Users\\21655\\bct_data_clean_2024_2025\\2024_Agregats_monnaie_contreparties_fr.pdf"

# Extraire les données
print("Extraction des données du PDF...")
df = extract_all_data_from_pdf(pdf_path)

# Sauvegarder dans un fichier CSV avec des virgules comme séparateurs décimaux
csv_path = "monetary_data_2001_2025_full_cleaned_fr.csv"
for col in df.columns:
    if col != 'index':
        if df[col].dropna().apply(lambda x: float(x).is_integer()).all():
            df[col] = df[col].astype('Int64')
        else:
            df[col] = df[col].astype(float)

df.reset_index().to_csv(csv_path, index=False, encoding='utf-8', sep=',', decimal=',', quoting=csv.QUOTE_NONE)

print(f"Fichier CSV généré : {csv_path}")

# Vérifier le CSV généré
print("\nVérification du CSV généré :")
df_check = pd.read_csv(csv_path, sep=',', decimal=',')
print("Types de données des colonnes dans le CSV :")
print(df_check.dtypes)
print("Colonnes dans le CSV :")
print(df_check.columns.tolist())
for col in df_check.columns:
    if col != 'index':
        non_numeric = df_check[col][~df_check[col].apply(lambda x: isinstance(x, (int, float)) or pd.isna(x))]
        if not non_numeric.empty:
            print(f"Valeurs non numériques dans la colonne '{col}' : {non_numeric}")

# Afficher le DataFrame avec le style
style = """
<style>
.table-bordered {
    border-collapse: collapse;
    width: 100%;
    font-family: Arial, sans-serif;
}
.table-bordered th, .table-bordered td {
    border: 1px solid #ddd;
    padding: 8px;
    text-align: center;
}
.table-bordered th {
    background-color: #f2f2f2;
    color: black;
}
.table-bordered tr:nth-child(even) {
    background-color: #f9f9f9;
}
.table-bordered tr:hover {
    background-color: #f1f1f1;
}
</style>
"""
print("Tableau complet avec bordures :")
html_table = style + df.to_html(index=True, border=1, classes='table table-bordered')
display(HTML(html_table))

Extraction des données du PDF...
Extract_all_data - Page 1 : 2 tableaux détectés
Extract_all_data - Tableau 1 extrait de la page 1 (nombre de lignes : 37)
Extract_all_data - Aperçu du tableau 1 : [['Indicateurs(cid:9)', '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06', '2025-07', '2025-08', '2025-09', '2025-10', '2025-11', '2025-12'], ['MASSE MONETAIRE M3', '132 826', '133 468', '', '', '', '', '', '', '', '', '', ''], ['MASSE MONETAIRE M2', '123 388', '123 715', '', '', '', '', '', '', '', '', '', ''], ['MASSE MONETAIRE M1', '56 520', '56 668', '', '', '', '', '', '', '', '', '', ''], ['MONNAIE FIDUCIAIRE', '22 139', '22 576', '', '', '', '', '', '', '', '', '', ''], ['MONNAIE SCRIPTURALE', '34 381', '34 092', '', '', '', '', '', '', '', '', '', ''], ['dt:Dépôts a vue auprès bques', '31 382', '31 083', '', '', '', '', '', '', '', '', '', ''], ['Dépôts à vue auprès du CCP', '2 913', '2 913', '', '', '', '', '', '', '', '', '', ''], ['M2 - M1', '66 867', '67 047', '', ''

,MASSE_MONETAIRE_M3,MASSE_MONETAIRE_M2,MASSE_MONETAIRE_M1,MONNAIE_FIDUCIAIRE,MONNAIE_SCRIPTURALE,dtDepôts_a_vue_auprès_bques,Depôts_a_vue_auprès_du_CCP,M2___M1,dt__Depôts_a_terme_et_autres_produits_financiers,Epargne_bancaire,Epargne_postale,M3___M2,dt__Certificats_de_depôt,Billets_de_tresorerie,PASSIFS_A_CACARTERE_NON_MONETAIRE,ACTIONS_ET_AUTRES_DE_PARTICIPATION,AUTRES_POSTES_NETS,TOTAL_RESSOURCES_=_TOTAL_CONTREPRTIES,AVOIRS_EXTERIEURS_NETS,Creances_sur_les_non_residents,Avoirs_exterieurs_de_la_BCT,Avoirs_exterieurs_des_AID,Engagements_envers_les_non_residents,Engagements_exter._de_la_BCT,Engagements_exter._des_AID,CREANCES_INTERIEURES,Creances_nettes_sur_l_Administration_Centrale,Creances_nettes_des_institutions_de_depôt,Creances_nettes_de_la_BCT,Creances_nettes_des_AID,Titres_de_l_Etat_aupres_public,C/P_des_depôts_CCP_et_CEP,moins_encaisse_du_Tresor,Creances_sur_l_economie,Credits_des_ID,Portefeuille_titres_des_ID
2001-12,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2002-01,17371,14164,6755,2361,4395,3904,484,7409,2026,4078,938,3207,798,341,1526,2484,-841,20539,1560,3186,2678,508,1627,328,1299,18979,1145,-448,26,-474,178,1422,-7,17834,16731,1103
2002-02,17295,14153,6654,2461,4194,3683,500,7498,2106,4083,953,3142,769,379,1412,2478,-754,20431,1346,2991,2495,496,1645,327,1318,19085,1213,-353,129,-483,124,1452,-10,17872,16723,1149
2002-03,17161,14080,6478,2418,4059,3574,478,7602,2151,4077,956,3081,646,368,1542,2480,-858,20324,1049,2675,2180,495,1625,338,1288,19275,1191,-362,152,-514,127,1434,-7,18084,16908,1175
2002-04,17059,13997,6446,2414,4033,3549,479,7550,2232,3940,964,3063,674,351,1663,2441,-719,20444,1741,3448,2933,515,1707,310,1398,18704,234,-1289,-749,-540,88,1443,-7,18469,17289,1180
2002-05,17172,14170,6452,2428,4024,3556,460,7718,2307,4043,967,3002,639,344,1537,2465,-756,20418,1932,3715,3156,559,1784,321,1463,18487,33,-1464,-833,-632,76,1426,-5,18454,17270,1184
2002-06,17529,14395,6602,2468,4134,3690,434,7793,2391,4034,966,3134,781,387,1453,2380,-585,20777,1918,3676,3091,585,1757,216,1542,18859,376,-1020,-198,-822,3,1400,-8,18483,17243,1240
2002-07,17493,14520,6706,2542,4164,3680,478,7813,2370,4044,960,2974,698,364,1411,2412,-445,20871,1911,3711,3046,665,1800,205,1595,18960,341,-1101,-215,-886,11,1438,-7,18619,17403,1216
2002-08,17535,14594,6664,2586,4078,3713,357,7930,2537,4022,957,2940,698,364,1441,2460,-427,21009,2020,3837,3169,668,1816,194,1623,18989,287,-1072,-154,-918,52,1314,-6,18702,17473,1229
2002-09,17504,14593,6556,2479,4077,3692,376,8037,2649,4041,960,2911,646,424,1484,2494,-341,21141,1899,3737,3080,656,1837,207,1630,19242,507,-883,0,-883,61,1336,-7,18734,17443,1291


In [76]:
pd.read_csv(csv_path)

,index,MASSE_MONETAIRE_M3,MASSE_MONETAIRE_M2,MASSE_MONETAIRE_M1,MONNAIE_FIDUCIAIRE,MONNAIE_SCRIPTURALE,dtDepôts_a_vue_auprès_bques,Depôts_a_vue_auprès_du_CCP,M2___M1,dt__Depôts_a_terme_et_autres_produits_financiers,...,Creances_nettes_sur_l_Administration_Centrale,Creances_nettes_des_institutions_de_depôt,Creances_nettes_de_la_BCT,Creances_nettes_des_AID,Titres_de_l_Etat_aupres_public,C/P_des_depôts_CCP_et_CEP,moins_encaisse_du_Tresor,Creances_sur_l_economie,Credits_des_ID,Portefeuille_titres_des_ID
0,2001-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-01,17371.0,14164.0,6755.0,2361.0,4395.0,3904.0,484.0,7409.0,2026.0,...,1145.0,-448.0,26.0,-474.0,178.0,1422.0,-7.0,17834.0,16731.0,1103.0
2,2002-02,17295.0,14153.0,6654.0,2461.0,4194.0,3683.0,500.0,7498.0,2106.0,...,1213.0,-353.0,129.0,-483.0,124.0,1452.0,-10.0,17872.0,16723.0,1149.0
3,2002-03,17161.0,14080.0,6478.0,2418.0,4059.0,3574.0,478.0,7602.0,2151.0,...,1191.0,-362.0,152.0,-514.0,127.0,1434.0,-7.0,18084.0,16908.0,1175.0
4,2002-04,17059.0,13997.0,6446.0,2414.0,4033.0,3549.0,479.0,7550.0,2232.0,...,234.0,-1289.0,-749.0,-540.0,88.0,1443.0,-7.0,18469.0,17289.0,1180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2025-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,2025-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,2025-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,2025-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import pandas as pd
import numpy as np

# Chemin vers le fichier CSV
csv_path = "C:\\Users\\21655\\Desktop\\monetary_data_2002_2025_full_cleaned_fr.csv"

# Lire le CSV
print(f"Lecture du fichier CSV : {csv_path}")
df = pd.read_csv(csv_path)

# Afficher les premières lignes pour un aperçu
print("\nAperçu des premières lignes du CSV :")
print(df.head())

# Afficher les dernières lignes pour vérifier 2025
print("\nAperçu des dernières lignes du CSV (période 2025) :")
print(df.tail(15))

# Vérifier les colonnes
print("\nColonnes détectées dans le CSV :")
print(df.columns.tolist())

# Vérifier les types de données de chaque colonne
print("\nTypes de données des colonnes :")
print(df.dtypes)

# Vérifier que toutes les colonnes (sauf 'index') contiennent uniquement des nombres ou NaN
print("\nVérification des colonnes pour des valeurs non numériques :")
for col in df.columns:
    if col == 'index':  # Ignorer la colonne 'index'
        print(f"Colonne '{col}' : Ignorée (colonne de dates)")
        continue
    
    # Vérifier si la colonne est numérique
    if not pd.api.types.is_numeric_dtype(df[col]):
        print(f"Erreur : La colonne '{col}' n'est pas numérique. Type détecté : {df[col].dtype}")
        print(f"Valeurs non numériques dans la colonne '{col}' :")
        # Identifier les valeurs non numériques
        non_numeric = df[col][~df[col].apply(lambda x: isinstance(x, (int, float)) or pd.isna(x))]
        print(non_numeric)
    else:
        print(f"Colonne '{col}' : OK (type numérique : {df[col].dtype})")
        # Vérifier les valeurs non-NaN pour s'assurer qu'elles sont bien des nombres
        non_nan_values = df[col].dropna()
        if not non_nan_values.empty:
            print(f"  Exemple de valeurs : {non_nan_values.head().tolist()}")

# Vérifier les lignes spécifiques pour 2025 (pour confirmer les NaN)
print("\nVérification des données pour 2025 :")
df_2025 = df[df['index'].str.startswith('2025')]
print(df_2025[['index', 'MASSE_MONETAIRE_M3', 'MASSE_MONETAIRE_M2', 'MASSE_MONETAIRE_M1']].tail(12))

# Résumé
print("\nRésumé :")
print(f"Nombre total de lignes : {len(df)}")
print(f"Nombre total de colonnes : {len(df.columns)}")
print("Si des valeurs non numériques sont détectées ci-dessus, elles doivent être corrigées avant l'importation dans Power BI.")

Lecture du fichier CSV : C:\Users\21655\Desktop\monetary_data_2002_2025_full_cleaned_fr.csv

Aperçu des premières lignes du CSV :
     index  MASSE_MONETAIRE_M3  MASSE_MONETAIRE_M2  MASSE_MONETAIRE_M1  \
0  2001-12                 NaN                 NaN                 NaN   
1  2002-01             17371.0             14164.0              6755.0   
2  2002-02             17295.0             14153.0              6654.0   
3  2002-03             17161.0             14080.0              6478.0   
4  2002-04             17059.0             13997.0              6446.0   

   MONNAIE_FIDUCIAIRE  MONNAIE_SCRIPTURALE  dtDepôts_a_vue_auprès_bques  \
0                 NaN                  NaN                          NaN   
1              2361.0               4395.0                       3904.0   
2              2461.0               4194.0                       3683.0   
3              2418.0               4059.0                       3574.0   
4              2414.0               4033.0        

In [19]:

# Chemin du fichier CSV
csv_path = "C:\\Users\\21655\\Downloads\\data.pfa1.csv"

# Étape 1 : Lire le fichier CSV avec pandas
try:
    df = pd.read_csv(csv_path)
    print("Fichier CSV chargé avec succès.")
except Exception as e:
    print(f"Erreur lors de la lecture du CSV : {e}")
    exit()

# Étape 2 : Se connecter à MongoDB
try:
    client = MongoClient("mongodb://localhost:27017/")
    print("Connexion à MongoDB réussie.")
except Exception as e:
    
    print(f"Erreur lors de la connexion à MongoDB : {e}")
    exit()

# Étape 3 : Créer ou sélectionner une base de données et une collection
db = client["monetary_db"]  # Base de données
collection = db["monetary_data"]  # Collection

# Étape 4 : Convertir le DataFrame en une liste de dictionnaires et insérer dans MongoDB
data = df.to_dict("records")
collection.insert_many(data)
print(f"{len(data)} documents insérés dans MongoDB.")

# Étape 5 : Fermer la connexion
client.close()
print("Connexion à MongoDB fermée.")



Fichier CSV chargé avec succès.
Connexion à MongoDB réussie.
278 documents insérés dans MongoDB.
Connexion à MongoDB fermée.
